In [1]:
import pandas as pd
import xlwings as xw
import openpyxl
import pyxlsb

print(f"{pd.__version__=}")
print(f"{xw.__version__=}")
print(f"{openpyxl.__version__=}")
print(f"{pyxlsb.__version__=}")

pd.__version__='1.5.0'
xw.__version__='dev'
openpyxl.__version__='3.0.10'
pyxlsb.__version__='1.0.9'


# Speed

### CSV

In [2]:
%%time
df_csv = pd.read_csv("AAPL.csv", index_col="Date", parse_dates=["Date"])

CPU times: user 11.6 ms, sys: 3.37 ms, total: 14.9 ms
Wall time: 19.6 ms


### pandas (via OpenPyXL)

In [3]:
%%time
df_xlsx = pd.read_excel("AAPL.xlsx", sheet_name=0, index_col="Date")

CPU times: user 673 ms, sys: 9.31 ms, total: 682 ms
Wall time: 710 ms


### xlwings

In [4]:
%%time
with xw.Book("AAPL.xlsx", mode="r") as book:
    sheet = book.sheets[0]
    df_xw = sheet.cells.options("df").value

CPU times: user 74.3 ms, sys: 4.83 ms, total: 79.2 ms
Wall time: 91.9 ms


### Check if all DataFrames are equal

In [5]:
df_xlsx.equals(df_csv) and df_xlsx.equals(df_xw)

True

# Even faster with xlsb!
=> Automatic DateTime conversion is currently missing (in both libraries, pandas and xlwings)

### pandas (via pyxlsb)

In [6]:
%%time
df_xlsb = pd.read_excel("AAPL.xlsb", sheet_name=0, index_col="Date")

CPU times: user 511 ms, sys: 4.65 ms, total: 515 ms
Wall time: 522 ms


### xlwings

In [7]:
%%time
with xw.Book("AAPL.xlsb", mode="r") as book:
    df_xlsb_xw = book.sheets[0].cells.options("df").value

CPU times: user 18.9 ms, sys: 3.9 ms, total: 22.8 ms
Wall time: 35 ms


### Check if all DataFrames are equal

In [8]:
df_xlsb.index = df_xlsb.index.astype(float)
df_xlsb_xw.equals(df_xlsb)

True

# Syntax

### pandas

In [9]:
df = pd.read_excel(
    "AAPL2.xlsx",
    sheet_name=0,
    skiprows=10,
    usecols="B:G",
    nrows=2,
    index_col="Date"
)
df

,Open,High,Low,Close,Adj Close
Date,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.405683
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.384517


### xlwings (A1 notation)

In [10]:
with xw.Book("AAPL2.xlsx", mode="r") as book:
    df = book.sheets[0]["B11:G13"].options("df").value
df

,Open,High,Low,Close,Adj Close
Date,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.405683
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.384517


### xlwings (slice notation)

In [11]:
with xw.Book("AAPL2.xlsx", mode="r") as book:
    sheet = book.sheets[0]
    df = sheet[10:13, 1:7].options("df").value
df

,Open,High,Low,Close,Adj Close
Date,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.405683
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.384517


# Named Ranges
(Not supported by pandas)

In [12]:
with xw.Book("AAPL2.xlsx", mode="r") as book:
    sheet = book.sheets[0]
    df = sheet["statistics"].options("df").value
df

,Value
Statistic,
Enterprise Value,2.42T
Trailing P/E,2.49T
Forward P/E,24.86


# Not everything is a DataFrame

In [13]:
with xw.Book("AAPL2.xlsx", mode="r") as book:
    sheet = book.sheets[0]
    myvalue = sheet["B2"].value
    mylist = sheet["B4:C4"].value
    # Dynamic range expansion!
    mydict = sheet["B5"].expand().options(dict).value

In [14]:
myvalue

'Apple Inc. (AAPL)'

In [15]:
mylist

['Statistic', 'Value']

In [16]:
mydict

{'Enterprise Value': '2.42T', 'Trailing P/E': '2.49T', 'Forward P/E': 24.86}